In [101]:
import pandas as pd
import gender_guesser.detector as gender
import string
import re
import numpy as np
import random
import ethnicolr
from ethnicolr import census_ln, pred_census_ln, pred_fl_reg_name, pred_fl_reg_ln

In [2]:
indiv = pd.read_csv('/Users/Brandon/Downloads/indiv18/by_date/itcont_2018_20180930_52010302.txt', sep = "|", nrows = 5000)

In [118]:
indiv.head()

,C00654095,A,YE,P,201802070200131475,15,IND,"DE LA PAZ, ALBERTO",LAS VEGAS,NV,...,"AGUAS, LLC",BUSINESS OWNER,11082018,500,Unnamed: 15,SA02091814752,1207802,X,Unnamed: 19,1060720180036821749
0,C00654095,A,YE,P,201802070200131527,15,IND,"DE LA PAZ, ALBERTO",LAS VEGAS,NV,...,"AGUAS, LLC",BUSINESS OWNER,11082018,1000,NaN,SA0209181527152,1207802,X,NaN,1060720180036821750
1,C00388157,N,YE,P,201801290300188510,15,IND,"BECKER, SCOTT",ASHTABULA,OH,...,CHROMAFLO TECHNOLOGIES CORP,PRESIDENT AND CEO,12282018,200,NaN,SA01311885104,1202723,NaN,NaN,2022120181514443536
2,C00616920,N,YE,P,201801250300187861,15,IND,"WATTS, RICHARD H",ST PETERSBURG,FL,...,SELF,ATTORNEY,11302019,250,NaN,SA013018786150,1201654,NaN,NaN,2022620181515929887
3,C00616920,N,YE,P,201801250300187848,15,IND,"BUCKLIN, ROBERT",NaN,FL,...,UNK,UNK,1205201,250,NaN,SA013018784813,1201654,NaN,NaN,2022620181515929850
4,C00338954,N,YE,P,201801319091220850,15E,IND,"PLATT, CAMERON",OAKLAND,CA,...,ABIO PROPERTIES,REAL ESTATE BROKER,10102018,500,C00484535,C10916238,1205084,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,4022220181514574567


In [127]:
###################################
# Extracting Sex From First Names #
###################################

rawNames = pd.DataFrame({'original_name':indiv['DE LA PAZ, ALBERTO']}) 

def clean_names(rawTable, colName):
    table = rawTable
    table['people'] = rawTable[rawTable[colName].str.contains(',') == True]
    table['last_name'] = table['people'].str.split(',').str[0]
    table['first_name'] = table['people'].str.split(',').str[1]
    table['first_name'] = table['first_name'].str.strip()
    table['last_name'] = table['last_name'].str.strip()
    table['first_name'] = table['first_name'].str.split(' ').str[0]
    table['first_name'].fillna('Entity', inplace = True) # minor compromise unknowns are male. randomize later. Not many unknowns at all
    table['last_name'].fillna('Entity', inplace = True)
    table['people'].fillna('Entity', inplace = True)
    table['first_name'] = table['first_name'].str.title()
    table['last_name'] = table['last_name'].str.title()
    return table


def binarized_gender_table(rawTable, colName):
    genderDict = {'male':1, 'mostly_male':1, 'female':0, 'mostly_female':0, 'unknown':1}
    det = gender.Detector()
    table = clean_names(rawTable, colName)
    table['sex'] = table['first_name'].apply(det.get_gender)
    table['sex'] = table['sex'].map(genderDict)
    return table


cleaned = binarized_gender_table(rawNames, 'original_name')
cleaned.head(20)

,original_name,people,last_name,first_name,sex
0,"DE LA PAZ, ALBERTO","DE LA PAZ, ALBERTO",De La Paz,Alberto,1.0
1,"BECKER, SCOTT","BECKER, SCOTT",Becker,Scott,1.0
2,"WATTS, RICHARD H","WATTS, RICHARD H",Watts,Richard,1.0
3,"BUCKLIN, ROBERT","BUCKLIN, ROBERT",Bucklin,Robert,1.0
4,"PLATT, CAMERON","PLATT, CAMERON",Platt,Cameron,1.0
5,KOCH INDUSTRIES INC.,Entity,Entity,Entity,1.0
6,"STONE, ROGER W.","STONE, ROGER W.",Stone,Roger,1.0
7,"HAIT, JANE","HAIT, JANE",Hait,Jane,0.0
8,"KANG, FELICIA","KANG, FELICIA",Kang,Felicia,0.0
9,"KING, KAREN","KING, KAREN",King,Karen,0.0


In [125]:
cleaned[cleaned['people'] == 'Entity']

,original_name,people,last_name,first_name,sex
5,KOCH INDUSTRIES INC.,Entity,Entity,Entity,1.0
20,NATIONAL RIGHT TO LIFE COMMITTEE,Entity,Entity,Entity,1.0
3279,HUMAN RIGHTS CAMPAIGN,Entity,Entity,Entity,1.0
4280,NATIONAL FOOTBALL LEAGUE,Entity,Entity,Entity,1.0
4281,NATIONAL FOOTBALL LEAGUE,Entity,Entity,Entity,1.0


In [231]:
len(test[test['gender'] == 1]) / len(test[test['gender'] == 0])

2.7802281368821293

In [126]:
##################################
# Extracting Ethnicity From Name #
##################################

# pred_fl_reg_name(cleaned, fname_col = 'first_name', lname_col = 'last_name')
test = cleaned.last_name.str.lower().to_frame()

names = [{'name': 'smith'}, {'name': 'zhang'}, {'name': 'jackson'}]
df = pd.DataFrame(names)

# pred_census_ln(df, 'name')
# census_ln(df, 'name')

census = census_ln(cleaned, 'last_name', 2010)
# ethnicolr.pred_census_ln(df, 'name')
# census['pctwhite'].isna()
census.head(10)

,original_name,people,last_name,first_name,sex,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,"DE LA PAZ, ALBERTO","DE LA PAZ, ALBERTO",De La Paz,Alberto,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,"BECKER, SCOTT","BECKER, SCOTT",Becker,Scott,1.0,94.89,0.52,0.66,0.34,1.26,2.33
2,"WATTS, RICHARD H","WATTS, RICHARD H",Watts,Richard,1.0,68.62,25.52,0.48,0.77,2.35,2.26
3,"BUCKLIN, ROBERT","BUCKLIN, ROBERT",Bucklin,Robert,1.0,94.38,(S),0.73,(S),2.08,2.14
4,"PLATT, CAMERON","PLATT, CAMERON",Platt,Cameron,1.0,86.57,7.75,0.49,0.55,2.09,2.55
5,KOCH INDUSTRIES INC.,Entity,Entity,Entity,1.0,NaN,NaN,NaN,NaN,NaN,NaN
6,"STONE, ROGER W.","STONE, ROGER W.",Stone,Roger,1.0,85.95,8.13,0.65,0.95,1.93,2.39
7,"HAIT, JANE","HAIT, JANE",Hait,Jane,0.0,92.01,(S),4.15,(S),(S),(S)
8,"KANG, FELICIA","KANG, FELICIA",Kang,Felicia,0.0,3.08,0.56,93.86,0.03,1.81,0.67
9,"KING, KAREN","KING, KAREN",King,Karen,0.0,70.16,22.76,1.23,0.98,2.31,2.55
